In [1]:
# Откройте Anaconda Prompt и выполните следующую команду

# (для Windows)
# pip install psycopg2

# (для MacOS)
# pip install psycopg2-binary

# Создаем соединение c базой данных (ВНИМАНИЕ! Ваши данные для подключения с БД могут отличаться)

# Импортируем драйвер для postgresql
import psycopg2

# Прячем пароль, чтобы все было безопасно
import getpass

# Задаем переменную для установления соединения с базой данных
conn = psycopg2.connect(user='postgres',database='postgres',host='localhost',port='5432',password=getpass.getpass('Введите пароль: '))

Введите пароль: ········


In [2]:
# Импортируем pandas для работы с табличными данными
import pandas as pd
# Присваеваем переменной текст запроса, который содержит необходимые данные для дальнейшей работы
query = "DROP TABLE if exists public.dz2; CREATE TABLE public.dz2 (grade text, change text, date date, ent_name text, rat_id smallint) TABLESPACE pg_default; ALTER TABLE public.dz2 OWNER to postgres; INSERT INTO dz2 SELECT query1.grade, change, date, ent_name, query1.rat_id FROM (SELECT grade, change, date, ent_name, actions.rat_id FROM actions JOIN rat_info ON actions.rat_id = rat_info.rat_id WHERE agency_id = 'EXP') as query1 JOIN scale_exp ON query1.grade = scale_exp.grade WHERE change <> 'снят' AND change <> 'приостановлен'; INSERT INTO dz2 (grade, date, ent_name) SELECT grade, date, ent_name FROM events WHERE grade = 'D'; SELECT * FROM dz2;"
# Передаем запрос в базу данных и записываем результат в переменную
zapros = pd.read_sql_query(query,conn)
# Выводим на экран первые 10 строчек, чтобы убедиться, что результат соответствует ожиданиям
zapros.head(10)


,grade,change,date,ent_name,rat_id
0,A,повышен,2011-05-24,АК Банк,14.0
1,B++,подтвержден,2013-08-23,Нефтяной Альянс КБ,14.0
2,B++,подтвержден,2014-01-23,РТБК КБ,14.0
3,A+,установлен,2012-07-06,СИБЭКО,16.0
4,A,подтвержден,2013-02-22,Синко-Банк КБ,14.0
5,A,повышен,2012-12-20,Финпромбанк АКБ,14.0
6,A+,установлен,2014-03-06,Хорус Финанс,16.0
7,C++,понижен,2015-12-03,1-й НПФ,27.0
8,B+,понижен,2015-10-06,1-й НПФ,27.0
9,A,понижен,2015-09-02,1-й НПФ,27.0


In [3]:
def my_fun(rat_id, period, matrix_type):
#     Сортируем данные из запроса
    kod = zapros.sort_values(by = ['grade', 'ent_name', 'date'], ascending = [True, True, True])
#     Ставим фильтр на определенный тип рейтинга и период времени
    kod = kod[kod['rat_id'] == rat_id]
    kod[(kod['date'] >= period[0])&(kod['date'] <= period[1])]
#     Переводим время в строковый формат, так как в запросах будем использовать его
    period[0] = str(period[0])
    period[1] = str(period[1])
#     Создаем массив, содержащий все виды рейтингов, которые удовлетворяют типу рейтинга и периоду времени
    kod1 = kod['grade'].unique()
#     Чтобы узнать, сколько видов рейтингов имеется, узнаем количество строк массива
    x = kod1.shape
#     Создаем переменную, списки и словари, которые далее будем использовать в циклах
    lll = 0
    yyy = []
    sss = {}
    sho = {}
    aaa = []
    bbb = []
    kk = []
#     Создаем условие, которое поможет вывести на экран определенный тип матрицы
    if matrix_type == 1:
#         Создаем цикл, который перебирает значения от 0 до количества видов рейтингов
        for i in range (0, x[0]):
#             Создаем запрос, который нам выдает таблицу с актуальными рейтингами на начало периода для каждого вида рейтинга
            c = "select ent_name, grade, assign_date from public.dz2 inner join (select max(date) as assign_date, ent_name as ent2_name from public.dz2 where (rat_id = " + str(rat_id) + " OR rat_id is NULL) AND date <= '" + period[0] + "' group by ent_name) as zapros1 on public.dz2.date = zapros1.assign_date and public.dz2.ent_name = zapros1.ent2_name where change <> 'снят' and change <> 'приостановлен' and (rat_id = " + str(rat_id) + " OR rat_id is null) and grade = '" + kod1[i] + "' ;"
#             Передаем запрос в SQL и загружаем результат в переменную c_q
            c_q = pd.read_sql_query(c,conn)
#             Считаем количество фирм на начало периода для каждого вида рейтинга и записываем результат в переменную zz
            zz = c_q['ent_name'].value_counts().sum()
#             Записываем в переменную pp вид рейтинга в данной итерации
            pp = kod1[i]
#             Вставляем фильтр, который убирает виды рейтинга, для которых на начало периода не было ни одной компании
            if zz != 0:
#                 Создаем цикл, который позволит отследить движения между видами рейтингов
                for j in range (0, x[0]):
#                     Создаем запрос, который нам выдает таблицу, в которой отслеживаются движения между рейтингами за заданный период
                    t = "select lol.ent_name, lol.grade, kek.grade as grade2 from (select ent_name, grade, assign_date from public.dz2 inner join (select max(date) as assign_date, ent_name as ent2_name from public.dz2 where (rat_id = " + str(rat_id) + " OR rat_id is null) AND date <= '" + period[0] + "' group by ent_name) as zapros1 on public.dz2.date = zapros1.assign_date and public.dz2.ent_name = zapros1.ent2_name where change <> 'снят' and change <> 'приостановлен' and (rat_id = " + str(rat_id) + " OR rat_id is null) and grade = '" + kod1[i] + "') as lol join (select ent_name, grade, assign_date from public.dz2 inner join (select max(date) as assign_date, ent_name as ent2_name from public.dz2 where (rat_id = " + str(rat_id) + " OR rat_id is null) AND date <= '" + period[1] + "' group by ent_name) as zapros1 on public.dz2.date = zapros1.assign_date and public.dz2.ent_name = zapros1.ent2_name where change <> 'снят' and change <> 'приостановлен' and (rat_id = " + str(rat_id) + " OR rat_id is null) and grade = '" + kod1[j] + "') as kek on lol.ent_name = kek.ent_name;"
#                     Передаем запрос в SQL и загружаем результат в переменную t_q
                    t_q = pd.read_sql_query(t,conn)
#                     Считаем количество фирм, перешедших из рейтинга i в рейтинг j, и записываем результат в переменную tt
                    tt = t_q['ent_name'].value_counts().sum()
#                     Находим вероятность того, что фирма перейдет из рейтинга i в рейтинг j за данный период
                    tt = tt/zz
#                     Округляем полученное значение до трех знаков после запятой
                    tt = round(tt, 3)
#                     Записываем в переменную вид рейтинга, соответствующий j
                    oo = kod1[j]
#                     Добавляем в список вероятность перехода для рейтинга i в рейтинг j
                    aaa.append(tt)
#                     Ограничиваем количество элементов в списке и добавляем в него название рейтинга j
                    if len(bbb) < x[0]:
                        bbb.append(oo)
#                 Добавляем в словарь список, содержащий вероятность перехода для рейтинга i в рейтинг j
                sho[kod1[i]] = aaa
#                 Обнуляем список для дальнейшей корректной работы
                aaa = []
#         Создаем Dataframe из полученное списка    
        vol = pd.DataFrame(sho)
#         Транспонируем полученную таблицу
        vol = vol.T
#         Называем столбцы видами рейтингов
        vol.columns = bbb
#         Создаем переменную, содержащую результат выполнения запроса
        results = {'Тип рейтинга': rat_id, 'Период времени': period, 'Матрица': 'переходных вероятностей когортным методом'}
#         Возвращаем входные данные и результат
        return results, vol
#     Создаем условие, которое поможет вывести на экран определенный тип матрицы
    elif matrix_type == 2:
#         Создаем цикл, который перебирает значения от 0 до количества видов рейтингов
        for i in range (0, x[0]):
#             Создаем запрос, который выдает нам для выбранного вида рейтинга все фирмы (в том числе с повторениями) за заданный период
            kak = "DROP TABLE if exists public.aaa; CREATE TABLE public.aaa (ent_name text, grade text, date date) TABLESPACE pg_default; ALTER TABLE public.aaa OWNER to postgres; insert into aaa (ent_name, grade, date) select ent_name, grade, assign_date from (select ent_name, grade, assign_date from public.actions inner join (select max(date) as assign_date, ent_name as ent2_name from public.actions where rat_id = " + str(rat_id) + " AND date <= '" + period[0] + "' and grade = '" + kod1[i] + "' group by ent_name) as zapros1 on public.actions.date = zapros1.assign_date and public.actions.ent_name = zapros1.ent2_name where change <> 'снят' and change <> 'приостановлен' and rat_id = " + str(rat_id) + " and grade = '" + kod1[i] + "') as kisa;insert into aaa (ent_name, grade, date) select ent_name, grade, date from dz2 where date >= '" + period[0] + "' and date <= '" + period[1] + "' and rat_id = " + str(rat_id) + " and grade = '" + kod1[i] + "'; select * from aaa;"
#             Передаем запрос в базу данных и записываем результат в переменную
            kak = pd.read_sql_query(kak, conn)
#             Передаем в переменную количество строк и столбцов (нас интересует количество строк)
            kak = kak.shape
#             Записываем в переменную общее количество фирм, побывавших в рейтинге i за данный период
            ii = kak[0]
#             Ставим фильтр, чтобы исключить те рейтинги, в которых не было ни одной фирмы за период
            if ii != 0:
#                 Задаем цикл, который позволит отследить движения между видами рейтингов
                for j in range (0, x[0]):
#                     Создаем запрос, результатом которого будет являться таблица, содержащая все изменения рейтингов за заданный период
                    vot = "DROP TABLE if exists public.aaa; CREATE TABLE public.aaa (ent_name text, grade text, date date) TABLESPACE pg_default; ALTER TABLE public.aaa OWNER to postgres; insert into aaa (ent_name, grade, date) select ent_name, grade, assign_date from (select ent_name, grade, assign_date from public.actions inner join (select max(date) as assign_date, ent_name as ent2_name from public.actions where rat_id = " + str(rat_id) + " AND date <= '" + period[0] + "' group by ent_name) as zapros1 on public.actions.date = zapros1.assign_date and public.actions.ent_name = zapros1.ent2_name where change <> 'снят' and change <> 'приостановлен' and rat_id = " + str(rat_id) + ") as kisa;insert into aaa (ent_name, grade, date) select ent_name, grade, date from dz2 where date >= '" + period[0] + "' and date <= '" + period[1] + "' and rat_id = " + str(rat_id) + "; select * from aaa;"
#                     Передаем запрос в базу данных и записываем результат в переменную
                    vot = pd.read_sql_query(vot, conn)
#                     Сортируем значения таким образом, чтобы далее их было удобно извлекаьть
                    vot = vot.sort_values(by = ['ent_name', 'date', 'grade'], ascending = [True, True, True])
#                     Меняем индексы в таблице
                    vot.index = range(vot.shape[0])
#                     Записываем в переменную количество строк и столбцов в таблице
                    rtr = vot.shape
#                     Задаем цикл, с помощью которого отслеживаем прямые переходы из рейтинга i в рейтинг j
                    for k in range (0, rtr[0]-1):
#                         С помощью данного условия как раз и задаем условие учета прямого перехода
                        if (vot.iloc[k]['ent_name'] == vot.iloc[k+1]['ent_name']) & (vot.iloc[k]['grade'] == kod1[i]) & (vot.iloc[k+1]['grade'] == kod1[j]):
#                             Если прямой переход из рейтинга i в рейтинг j наблюдался, то прибавляем единицу к переменной, которая изначально была равна 0
                            lll = lll + 1
#                     Делим полученное число переходов на общее количество фирм, побывавших с рейтингов i в данном периоде
                    lll = lll/ii
#                     Округляем полученное значение до 3 знаков после запятой
                    lll = round(lll, 3)
#                     Добавляем полученное значение в список
                    yyy.append(lll)
#                     Обнуляем переменную для дальнейшей корректной работы
                    lll = 0
#                     Записываем в переменную имя рейтинга j
                    oo = kod1[j]
#                     Задаем ограничение на длину формируемого списка
                    if len(bbb) < x[0]:
#                         Передаем имя всех задействованных рейтингов в список
                        bbb.append(oo)
#                 Передаем сформированный список в словарь
                sss[kod1[i]] = yyy
#                 Обнуляем спсок для дальнейшей корректной работы
                yyy = []
#         Полученный словарь преобразовываем в DataFrame
        ura = pd.DataFrame(sss)
#         Транспонируем полученную таблицу
        ura = ura.T
#         Меняем названия столбцов
        ura.columns = bbb
#         Передаем входные данные в словарь
        results = {'Тип рейтинга': rat_id, 'Период времени': period, 'Матрица': 'интенсивностей методом дюраций'}
#         Возвращаем входные данные и результат
        return results, ura
    else:
        print('Вы неправильно ввели тип матрицы :(')

In [4]:
# Импортируем datetime для перевода текстового формата в дату
import datetime as dt
# Вводим тип рейтинга
rat_id = int(input('Введите тип рейтинга (для агенства Эксперт РА это число от 12 до 29): '))
# Создаем список, в котором будет содержаться рассматриваемый период
period = []
# Вводим дату начала периода
date1 = input('Введите дату начала периода в формате ГГГГ-ММ-ДД: ')
# Переводим дату в правильный формат
date1 = dt.datetime.strptime(date1, '%Y-%m-%d').date()
# Добавляем начало периода в список
period.append(date1)
# Вводим дату окончания периода
date2 = input('Введите дату окончания периода в формате ГГГГ-ММ-ДД: ')
# Переводим дату в правильный формат
date2 = dt.datetime.strptime(date2, '%Y-%m-%d').date()
# Добавляем окончание периода в список
period.append(date2)
# Вводим тип матрицы, который хотим видеть в итоге
matrix_type = int(input('Выберите тип матрицы для вывода (1 - матрица переходных вероятностей; 2 - матрица интенсивностей): '))
# Запрашиваем исполнение функции
my_fun(rat_id, period, matrix_type)

Введите тип рейтинга (для агенства Эксперт РА это число от 12 до 29): 20
Введите дату начала периода в формате ГГГГ-ММ-ДД: 2014-05-16
Введите дату окончания периода в формате ГГГГ-ММ-ДД: 2018-03-08
Выберите тип матрицы для вывода (1 - матрица переходных вероятностей; 2 - матрица интенсивностей): 2


({'Тип рейтинга': 20,
  'Период времени': ['2014-05-16', '2018-03-08'],
  'Матрица': 'интенсивностей методом дюраций'},
          A     A+  A++      B     B+    B++    C++      E
 A    0.430  0.013  0.0  0.025  0.013  0.228  0.000  0.025
 A+   0.200  0.467  0.0  0.000  0.000  0.000  0.000  0.000
 A++  0.000  0.000  0.0  0.000  0.000  0.000  0.000  0.000
 B    0.000  0.000  0.0  0.333  0.000  0.000  0.000  0.000
 B+   0.000  0.000  0.0  0.000  0.200  0.000  0.000  0.000
 B++  0.042  0.000  0.0  0.042  0.125  0.167  0.042  0.042
 C++  0.000  0.000  0.0  0.000  0.000  0.000  0.000  0.000
 E    0.000  0.000  0.0  0.000  0.000  0.000  0.000  0.000)